In [25]:
#imports
import math
from PIL import Image, ExifTags
import piexif

In [ ]:
# Convert image from .HEIC to jpg
# Step 1: Open HEIC and extract metadata
heif_file = pillow_heif.open_heif("blackpig.HEIC", convert_hdr_to_8bit=False, bgr_mode=False)
image = heif_file[0].to_pillow()
exif_data = heif_file[0].info.get("exif", None)

# Step 2: Save image as JPEG
image.save("blackpig.jpg", format="JPEG")

# Step 3: Write metadata to JPEG (if available)
if exif_data:
    piexif.insert(exif_data, "blackpig.jpg")

In [26]:
# check if the picture have meta data
img = Image.open('bottle.jpg')
exif_data = img._getexif()
if exif_data:
    print("EXIF data found.")
else:
    print("No EXIF data found.")

EXIF data found.


In [ ]:
# print meta data
from PIL.ExifTags import TAGS

def print_exif(image_path):
    with Image.open(image_path) as img:
        exif_data = img._getexif()
        if exif_data is None:
            print("No EXIF data found.")
            return
        for tag_id, value in exif_data.items():
            tag = TAGS.get(tag_id, tag_id)
            print(f"{tag}: {value}")

if __name__ == "__main__":
    image_path = "bottle.jpg"  # Change to your image file path
    print_exif(image_path)

ImageWidth: 4000
ImageLength: 3000
ResolutionUnit: 2
ExifOffset: 228
Make: samsung
Model: Galaxy S24+
Software: S926BXXS8BYG1
Orientation: 6
DateTime: 2025:09:12 11:52:26
YCbCrPositioning: 1
XResolution: 72.0
YResolution: 72.0
ExifVersion: b'0220'
ShutterSpeedValue: 0.01
ApertureValue: 1.69
DateTimeOriginal: 2025:09:12 11:52:26
DateTimeDigitized: 2025:09:12 11:52:26
BrightnessValue: 3.37
ExposureBiasValue: 0.0
MaxApertureValue: 1.69
MeteringMode: 3
Flash: 0
FocalLength: 5.4
ColorSpace: 65535
ExifImageWidth: 4000
DigitalZoomRatio: 1.0
FocalLengthIn35mmFilm: 23
SceneCaptureType: 0
OffsetTime: +07:00
OffsetTimeOriginal: +07:00
SubsecTime: 106
SubsecTimeOriginal: 106
SubsecTimeDigitized: 106
ExifImageHeight: 3000
ExposureTime: 0.01
FNumber: 1.8
ImageUniqueID: H50XLQE00SM
ExposureProgram: 2
ISOSpeedRatings: 100
ExposureMode: 0
FlashPixVersion: b'0100'
WhiteBalance: 0


In [30]:
def rational_to_float(r):
    """Convert EXIF rational (num, denom) to float."""
    if isinstance(r, tuple) and len(r) == 2 and r[1] != 0:
        return r[0] / r[1]
    elif isinstance(r, (int, float)):
        return float(r)
    return None

def get_exif_info(image_path):
    """Extract useful EXIF metadata from an image."""
    img = Image.open(image_path)
    exif_bytes = img.info.get("exif", None)
    if not exif_bytes:
        return None

    exif_dict = piexif.load(exif_bytes)

    # Extract values
    focal_length = rational_to_float(exif_dict["Exif"].get(piexif.ExifIFD.FocalLength))
    focal_35 = exif_dict["Exif"].get(piexif.ExifIFD.FocalLengthIn35mmFilm)
    if focal_35:
        focal_35 = int(focal_35)

    model = exif_dict["0th"].get(piexif.ImageIFD.Model, b"").decode(errors="ignore")
    make = exif_dict["0th"].get(piexif.ImageIFD.Make, b"").decode(errors="ignore")

    width_px, height_px = img.size

    return {
        "focal_length_mm": focal_length,
        "focal_length_35mm": focal_35,
        "model": model,
        "make": make,
        "image_width_px": width_px,
        "image_height_px": height_px,
    }

def estimate_sensor_width(focal_length_mm, focal_length_35mm):
    """
    Estimate sensor width (mm) using crop factor if actual sensor size is unknown.
    - crop_factor = focal_35mm / focal_mm
    - sensor_width = 36 mm / crop_factor   (based on full-frame width = 36 mm)
    """
    if not focal_length_mm or not focal_length_35mm:
        return None
    crop_factor = focal_length_35mm / focal_length_mm
    return 36.0 / crop_factor
    
def compute_object_length(pixel_length, image_width_px, focal_length_mm, sensor_width_mm, distance_mm):
    """
    Compute real-world object length in mm.
    Formula:
    - hfov = 2 * atan(sensor_width / (2 * focal_length))
    - physical_width = 2 * distance * tan(hfov/2)
    - mm_per_pixel = physical_width / image_width_px
    - object_length_mm = pixel_length * mm_per_pixel
    """
    if not all([pixel_length, image_width_px, focal_length_mm, sensor_width_mm, distance_mm]):
        raise ValueError("Missing required parameters.")

    hfov = 2.0 * math.atan(sensor_width_mm / (2.0 * focal_length_mm))
    physical_width_mm = 2.0 * distance_mm * math.tan(hfov / 2.0)
    mm_per_pixel = physical_width_mm / image_width_px
    return pixel_length * mm_per_pixel

# Path to your photo
image_path = "blackpig.jpg"

# Step 1: Extract EXIF
exif_info = get_exif_info(image_path)
print("EXIF info:", exif_info)

# Step 2: Get or estimate sensor width
sensor_width_mm = 7.6  # for iphone 15
if not sensor_width_mm and exif_info["focal_length_mm"] and exif_info["focal_length_35mm"]:
    sensor_width_mm = estimate_sensor_width(exif_info["focal_length_mm"], exif_info["focal_length_35mm"])
    print("Estimated sensor width:", sensor_width_mm)

# Step 3: Known measurement inputs
distance_mm = 4000.0       # distance camera -> object in mm (must measure or estimate) Estimate
pixel_length = 146         # object's pixel length (e.g., contour bounding box width)

# Step 4: Compute real-world length
real_length_mm = compute_object_length(
    pixel_length=pixel_length,
    image_width_px=exif_info["image_width_px"],
    focal_length_mm=exif_info["focal_length_mm"],
    sensor_width_mm=sensor_width_mm,
    distance_mm=distance_mm
)

print(f"Object length ≈ {real_length_mm:.2f} mm ({real_length_mm/10:.2f} cm)")

EXIF info: {'focal_length_mm': 6.764999866370901, 'focal_length_35mm': 24, 'model': 'iPhone 15 Pro Max', 'make': 'Apple', 'image_width_px': 4284, 'image_height_px': 5712}
Object length ≈ 153.15 mm (15.31 cm)
